In [1]:
import json
import argparse
from pprint import pprint
import random
import numpy as np

In [2]:
train_file_A = '../datasets/naps/naps.trainA.1.0.jsonl' # 5.9G
train_file_B = '../datasets/naps/naps.trainB.1.0.jsonl'
test_file = '../datasets/naps.test.1.0.jsonl'

In [3]:
def get_by_key(example, key):
    if isinstance(example[key], str):
        ch = " " if key in ['text', 'code_sequence'] else ""
        return ch.join(example[key])
    else:
        return example[key]

def get_all_by_key(data, key):
    return [get_by_key(d, key) for d in data]

In [4]:
lines = [l.strip() for l in open(train_file_B, "rt").readlines()]
data = [dict(json.loads(l)) for l in lines]

keys = list(data[0].keys())
keys

['solution_id',
 'problem_id',
 'is_train',
 'return_type',
 'url',
 'is_partial',
 'passes_tests',
 'entry_id',
 'code_sequence',
 'tests',
 'text',
 'language',
 'args',
 'code_tree']

In [5]:
idx = 12

" ".join(get_by_key(data[idx], key='text'))

'given integers var3 , var4 . initialize var5 to 0 , var2 to 62 . while var3 - th bit of var2 equals to var4 - th bit of var2 , decrease var2 by 1 .'